##1. Import library 

In [1]:
import torch
import torch.nn as nn 
import numpy as np 
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()


True

##2. Data preprocessing

In [2]:
import os 
import glob 
from PIL import Image 
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
from torchvision.models import resnet18, ResNet18_Weights

In [4]:
weights = ResNet18_Weights.DEFAULT
resnet_model = resnet18(weights = weights)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 185MB/s]


In [5]:
resnet_model.requires_grad_(False)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
train_path = '/kaggle/input/train-dataset/train'
valid_path = '/kaggle/input/valid-data/test'

In [68]:
img_width, img_height = 224,224
random_trans = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),          
    transforms.Normalize(           
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]),
    transforms.CenterCrop(224)
])

In [22]:
data_labels = ['freshapples','freshbanana','freshoranges','rottenapples','rottenbanana','rottenoranges']

In [23]:
class MyDataset(Dataset):
    def __init__(self,data_dir):
        self.imgs = []
        self.labels = [] 
        
        for idx, label in enumerate(data_labels):
            folder_path = os.path.join(data_dir,label)
            data_paths = glob.glob(os.path.join(folder_path,'*.png'),recursive = True)
            for path in data_paths:
                img = tv_io.read_image(path,tv_io.ImageReadMode.RGB)
                img = img.float() / 255.0
                self.imgs.append(random_trans(img).to(device))
                self.labels.append(torch.tensor(idx).to(device).float())

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self,idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img,label
        

In [24]:
train_data = MyDataset(train_path)
valid_data = MyDataset(valid_path)

train_loader = DataLoader(train_data,batch_size = 32, shuffle = True)
valid_loader = DataLoader(valid_data,batch_size = 32,shuffle = False)

In [25]:
print(len(train_loader))
print(len(valid_loader))

341
85


In [27]:
n_classes = len(data_labels)
print(n_classes)

6


In [70]:
resnet_model.fc = nn.Sequential(
    nn.Linear(512,512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512,n_classes)
)

In [71]:
model = resnet_model
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

##3. Build model 

In [72]:
model = model.to(device)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [85]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr = 1e-5,weight_decay = 1e-5)

In [86]:
train_N, valid_N = len(train_loader.dataset), len(valid_loader.dataset)

In [87]:
def get_batch_accuracy(output,y,N):
    pred = output.argmax(dim = 1,keepdim = True)
    accuracy = pred.eq(y.view_as(pred)).sum().item()
    return accuracy / N 
    

##4.Train and valid model

In [88]:
def train():
    loss = 0
    accuracy = 0
    model.train()
    for x,y in train_loader:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output,y.long())
        batch_loss.backward()
        optimizer.step()
        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output,y,train_N)
    print(f'Loss: {loss:.4f} Accuracy: {accuracy*100:.2f}%')

In [89]:
def valid():
    loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for x,y in valid_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            batch_loss = loss_function(output,y.long())
            loss += batch_loss.item()
            accuracy += get_batch_accuracy(output,y,valid_N)
    print(f'Loss: {loss:.4f} Accuracy: {accuracy*100:.2f}%')

In [92]:
def result():
    loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for x,y in valid_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            batch_loss = loss_function(output,y.long())
            loss += batch_loss.item()
            accuracy += get_batch_accuracy(output,y,valid_N)
    print(f'Accuracy: {accuracy*100:.2f}%')

In [81]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch:{epoch}/{num_epochs}:')
    train()
    valid()

Epoch:0/10:
Loss: 4.2070 Accuracy: 99.57%
Loss: 41.2165 Accuracy: 87.55%
Epoch:1/10:
Loss: 3.1320 Accuracy: 99.64%
Loss: 48.5917 Accuracy: 86.14%
Epoch:2/10:
Loss: 2.5473 Accuracy: 99.81%
Loss: 42.2010 Accuracy: 86.92%
Epoch:3/10:
Loss: 4.0297 Accuracy: 99.60%
Loss: 38.6845 Accuracy: 87.88%
Epoch:4/10:
Loss: 3.1771 Accuracy: 99.67%
Loss: 36.8238 Accuracy: 88.44%
Epoch:5/10:
Loss: 2.3088 Accuracy: 99.81%
Loss: 33.8279 Accuracy: 88.55%
Epoch:6/10:
Loss: 2.0684 Accuracy: 99.79%
Loss: 35.1935 Accuracy: 88.77%
Epoch:7/10:
Loss: 2.7711 Accuracy: 99.73%
Loss: 43.7913 Accuracy: 86.40%
Epoch:8/10:
Loss: 2.8511 Accuracy: 99.83%
Loss: 33.6683 Accuracy: 89.14%
Epoch:9/10:
Loss: 2.0571 Accuracy: 99.77%
Loss: 33.4538 Accuracy: 89.29%


In [83]:
resnet_model.requires_grad_(True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [90]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch:{epoch}/{num_epochs}:')
    train()
    valid()

Epoch:0/5:
Loss: 3.0195 Accuracy: 99.71%
Loss: 0.0146 Accuracy: 100.00%
Epoch:1/5:
Loss: 1.0377 Accuracy: 99.97%
Loss: 0.0143 Accuracy: 100.00%
Epoch:2/5:
Loss: 0.7778 Accuracy: 99.96%
Loss: 0.0151 Accuracy: 100.00%
Epoch:3/5:
Loss: 0.4411 Accuracy: 99.98%
Loss: 0.0611 Accuracy: 100.00%
Epoch:4/5:
Loss: 0.5312 Accuracy: 99.94%
Loss: 0.0315 Accuracy: 100.00%


In [93]:
result()

Accuracy: 100.00%
